# Encontrar a nuestros vendedores y productos con mejor desempeño

## Introduccion

**Contexto empresarial.** Trabaja para AdventureWorks, una empresa que vende equipos deportivos al aire libre. La compañía tiene muchas ubicaciones diferentes y ha estado registrando las ventas de diferentes ubicaciones en varios productos. A usted, su nuevo científico de datos, se le ha encomendado la siguiente pregunta: **"¿Cuáles son nuestros mejores productos y vendedores y cómo podemos utilizar esta información para mejorar nuestro rendimiento general?"**

Se le ha dado acceso a los archivos de datos relevantes con documentación del departamento de TI. Su trabajo consiste en extraer información valiosa de estos archivos de datos para ayudar a aumentar las ventas. Primero, observará los mejores productos y tratará de ver cómo se comportan los diferentes productos en diferentes categorías. En segundo lugar, analizará a los mejores vendedores para ver si el porcentaje de comisión los motiva a vender más.

**Problema comercial.** Su tarea es **construir una base de datos a partir de los archivos CSV proporcionados y luego escribir consultas en SQL para llevar a cabo el análisis solicitado**.

**Contexto analítico.** Recibirá los datos (almacenados en la carpeta ```data / csvs```) como un conjunto de archivos CSV separados, cada uno de los cuales representa una tabla. Construirá una nueva base de datos en formato SQL

La compañía ha sido bastante vaga sobre cómo esperan que extraiga información, pero ha elaborado el siguiente plan de ataque:

1. Cree la base de datos y asegúrese de poder ejecutar consultas básicas en ella.
2. Observa cómo se relacionan las calificaciones de los productos y las ventas totales.
3. Vea cómo se venden los productos en diferentes subcategorías (bicicletas, cascos, calcetines, etc.)
4. Calcule qué vendedores se desempeñaron mejor en el último año.
5. Ver si las ventas totales están correlacionadas con su porcentaje de comisión.

Por supuesto, este es solo su plan inicial. A medida que explora la base de datos, su estrategia cambiará.

## Overview of the data

La data que se encuentra dentro del directorio ```./data/csvs```; representa la data proporcionada por ```AdventureWorks``` . Nos enfocarmeos en las Ventas y las categorias de los productos. Mas informacion la encontrara en  [AQUI](https://dataedo.com/download/AdventureWorks.pdf). 

**Product Tables:**
* **Product**: one row per product that the company sells
* **ProductReview**: one row per rating and review left by customers
* **ProductModelProductDescriptionCulture**: a link between products and their longer descriptions also indicating a "culture" - which language and region the product is for
* **ProductDescription**: a longer description of each product, for a specific region
* **ProductCategory**: the broad categories that products fit into
* **ProductSubCategory**: the narrower subcategories that products fit into

**Sales Tables:**
* **SalesPerson**: one row per salesperson, including information on their commission and performance
* **SalesOrderHeader**: one row per sale summarizing the sale
* **SalesOrderDetail**: many rows per sale, detailing each product that forms part of the sale
* **SalesTerritory**: the different territories where products are sold, including performance

**Region Tables:**
* **CountryRegionCurrency**: the currency used by each region
* **CurrencyRate**: the average and closing exchange rates for each currency compared to the USD

## Creando la database y adiconando la data
Es necesario relaizar un procedimiento previo con la tabla productmodeldescriptionculture

In [8]:
import pandas as pd
productmodelproductdescriptionculture=pd.read_csv('data/csvs/productmodelproductdescriptionculture.csv')
#productmodelproductdescriptionculture['cultureid']=productmodelproductdescriptionculture['cultureid'].str.strip()
productmodelproductdescriptionculture.to_csv('data/csvs/productmodelproductdescriptionculture.csv', sep=",")  
productmodelproductdescriptionculture.head()

,Unnamed: 0,productmodelid,productdescriptionid,cultureid,modifieddate
0,0,1,1199,en,2013-04-30 00:00:00
1,1,1,1467,ar,2013-04-30 00:00:00
2,2,1,1589,fr,2013-04-30 00:00:00
3,3,1,1712,th,2013-04-30 00:00:00
4,4,1,1838,he,2013-04-30 00:00:00


In [9]:
productmodelproductdescriptionculture.drop(["Unnamed: 0"],axis=1, inplace=True)

In [10]:
productmodelproductdescriptionculture.head()

,productmodelid,productdescriptionid,cultureid,modifieddate
0,1,1199,en,2013-04-30 00:00:00
1,1,1467,ar,2013-04-30 00:00:00
2,1,1589,fr,2013-04-30 00:00:00
3,1,1712,th,2013-04-30 00:00:00
4,1,1838,he,2013-04-30 00:00:00


In [55]:
import pandas as pd
from sqlalchemy import create_engine, text

#maximum number of rows to display
pd.options.display.max_rows = 10

engine=create_engine('sqlite://')
df = pd.read_csv('data/csvs/product.csv').to_sql('product', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/productreview.csv').to_sql('productreview', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/productmodelproductdescriptionculture.csv').to_sql('productmodelproductdescriptionculture', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/productdescription.csv').to_sql('productdescription', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/salesorderdetail.csv').to_sql('salesorderdetail', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/productcategory.csv').to_sql('productcategory', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/productsubcategory.csv').to_sql('productsubcategory', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/salesperson.csv').to_sql('salesperson', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/salesorderheader.csv').to_sql('salesorderheader', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/salesterritory.csv').to_sql('salesterritory', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/countryregioncurrency.csv').to_sql('countryregioncurrency', engine, if_exists='replace', index=False)
df = pd.read_csv('data/csvs/currencyrate.csv').to_sql('currencyrate', engine, if_exists='replace', index=False)



def runQuery(sql):
    result = engine.connect().execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

## Encontrando los productos mas populares

Como se comentó, a la empresa le gustaría saber cuál de sus productos es el más popular entre los clientes. Calcula que la calificación promedio otorgada en las reseñas está correlacionada con el número de ventas de un producto en particular (que los productos con críticas más altas tienen más ventas).

### Ejercicio 1: (15 min)

Usando las tablas ```product``` y ```productreview```, ```JOIN``` y clasifica los productos de acuerdo con su calificación promedio de revisión. ¿Cuáles son los nombres y las identificaciones de los 5 productos principales?


**Respuesta**

In [43]:
query = \
"""
select pr.productid,p.name, avg(pr.rating) avg_rating from productreview pr JOIN product p ON p.productid = pr.productid GROUP BY pr.productid
"""
res = pd.read_sql(query,engine)

In [41]:
res

,productid,name,avg_rating
0,709,"Mountain Bike Socks, M",5.0
1,798,"Road-550-W Yellow, 40",5.0
2,937,HL Mountain Pedal,3.0


### Ejercicio 2: (30 min)

Para su decepción, ¡solo hay tres productos con calificaciones y solo cuatro reseñas en total! Esto no es lo suficientemente cerca como para realizar un análisis de la correlación entre las revisiones y las ventas totales.

Sin embargo, su gerente quiere la **descripción en inglés** de estos productos para una próxima venta. ¡Utilice la documentación proporcionada anteriormente si necesita ayuda para navegar por la estructura para extraer esto!

**Respuesta.**

In [53]:
query = \
"""
SELECT 
    p.productid 
    ,p.name 
    ,pmx.cultureid 
    ,pd.description 
FROM product p 
    INNER JOIN productmodelproductdescriptionculture pmx 
    ON p.productmodelid = pmx.productmodelid 
    INNER JOIN productdescription pd 
    ON pmx.productdescriptionid = pd.productdescriptionid
WHERE pmx.cultureid = "en"
"""
res = pd.read_sql(query,engine)
res

,productid,name,cultureid,description
0,864,"Classic Vest, S",en,"Light-weight, wind-resistant, packs to fit int..."
1,865,"Classic Vest, M",en,"Light-weight, wind-resistant, packs to fit int..."
2,866,"Classic Vest, L",en,"Light-weight, wind-resistant, packs to fit int..."
3,712,AWC Logo Cap,en,Traditional style with a flip-up brim; one-siz...
4,861,"Full-Finger Gloves, S",en,"Synthetic palm, flexible knuckles, breathable ..."
...,...,...,...,...
289,823,LL Mountain Rear Wheel,en,Replacement mountain wheel for entry-level rider.
290,824,ML Mountain Rear Wheel,en,Replacement mountain wheel for the casual to s...
291,825,HL Mountain Rear Wheel,en,High-performance mountain replacement wheel.
292,826,LL Road Rear Wheel,en,Replacement road rear wheel for entry-level cy...


### Ejercicio 3: (30 min)

Dado que no podemos inferir los productos más populares de las revisiones, optaremos por una estrategia alternativa.

Obtenga la ID del modelo, el nombre, la descripción y el número total de ventas de cada producto y muestre los 10 productos más vendidos. Puede inferir con qué frecuencia se han vendido los productos mirando la tabla `salesorderdetail` (cada fila puede indicar más de una venta, así que tome nota de` OrderQty`).

**Respuesta.**

In [62]:
query = \
"""
SELECT 
    p.productid 
    ,p.name
    , SUM(sod.orderqty) total_sales
FROM product p
INNER JOIN salesorderdetail sod
ON p.productid = sod.productid
GROUP BY p.productid
"""
res = pd.read_sql(query,engine)
res.sort_values(by="total_sales", ascending=False).head(10)

,productid,name,total_sales
5,712,AWC Logo Cap,8311
139,870,Water Bottle - 30 oz.,6815
4,711,"Sport-100 Helmet, Blue",6743
8,715,"Long-Sleeve Logo Jersey, L",6592
1,708,"Sport-100 Helmet, Black",6532
0,707,"Sport-100 Helmet, Red",6266
133,864,"Classic Vest, S",4247
142,873,Patch Kit/8 Patches,3865
153,884,"Short-Sleeve Classic Jersey, XL",3864
7,714,"Long-Sleeve Logo Jersey, M",3636


### Ejercicio 4: (30 min)

Calcule las cantidades totales de `salesorderdetail` nuevamente y agrupe los productos por subcategoría

**Respuesta.** 

In [63]:
query4="""
WITH product_qtys
AS (
    SELECT productid,
        SUM(orderqty) AS quantity
    FROM salesorderdetail
    GROUP BY productid
    ),
product_price_qty
AS (
    SELECT pc.name AS category,
        ps.name AS subcategory,
        p.listprice,
        sum(product_qtys.quantity) AS quantity
    FROM product p
    INNER JOIN product_qtys
        ON p.productid = product_qtys.productid
    INNER JOIN productsubcategory ps
        ON p.productsubcategoryid = ps.productsubcategoryid
    INNER JOIN productcategory pc
        ON ps.productcategoryid = pc.productcategoryid
    GROUP BY pc.name,
        ps.name,
        p.listprice
    )
SELECT subcategory,
    sum(quantity) AS total_qty
FROM product_price_qty ppq
GROUP BY subcategory
ORDER BY total_qty DESC limit 10;
"""

runQuery(query4)

,subcategory,total_qty
0,Road Bikes,47196
1,Mountain Bikes,28321
2,Jerseys,22711
3,Helmets,19541
4,Tires and Tubes,18006
5,Touring Bikes,14751
6,Gloves,13012
7,Road Frames,11753
8,Mountain Frames,11621
9,Bottles and Cages,10552


## Encontrar a nuestros mejores vendedores

Como se mencionó anteriormente, queremos encontrar a nuestros mejores vendedores y ver si podemos o no incentivarlos de manera adecuada. Es decir, queremos determinar si el porcentaje de comisión que les damos los motiva a realizar más y mayores ventas.

### Ejercicio 5: (10 min)

Encuentre los cinco vendedores con mejor desempeño usando la columna `salesytd` (Sales, year-to-date). (Solo necesitamos conocer el `businessentityid` de cada vendedor, ya que esto identifica de forma única a cada uno). ¿Por qué podría ser escéptico con estos números en este momento?

**Respuesta.**

In [65]:
query = \
"""
SELECT
sp.businessentityid,
sp.salesytd
FROM salesperson sp
ORDER BY salesytd DESC
LIMIT 10
"""
res = pd.read_sql(query,engine)
res

,businessentityid,salesytd
0,276,4.251369e+06
1,289,4.116871e+06
2,275,3.763178e+06
3,277,3.189418e+06
4,290,3.121616e+06
5,282,2.604541e+06
6,281,2.458536e+06
7,279,2.315186e+06
8,288,1.827067e+06
9,284,1.576562e+06


In [ ]:
## SUM(ValorProduct * Cantidad) per Vendedor

Los números están codificados en esta tabla, en lugar de calcularse dinámicamente a partir de cada registro de ventas. Actualmente, no sabemos cómo se actualiza este número o mucho al respecto, por lo que es bueno permanecer escéptico.

### Ejercicio 6: (15 min)

Usando ```salesorderheader```, busque los 5 mejores vendedores que hicieron la mayor cantidad de ventas **en el año más reciente** (2014). (Hay una columna llamada `subtotal`; úsela). Las ventas que no tienen un vendedor asociado deben excluirse de sus cálculos y producción final. Se deben incluir todos los pedidos que se realizaron dentro del año calendario 2014.

**Pista:** Puedes usar la sintaxis `'1970-01-01'` para generar un punto de comparacion en el tiempo

**Respuesta.**

In [77]:
query = \
"""
SELECT
soh.salespersonid,
SUM(soh.subtotal) as sum_subtotal
FROM salesorderheader soh
WHERE salespersonid IS NOT NULL AND orderdate BETWEEN '2014-01-01' AND '2014-12-31'
GROUP BY soh.salespersonid
"""
res = pd.read_sql(query,engine)
res.sort_values(by="sum_subtotal", ascending=False).head(5)

,salespersonid,sum_subtotal
15,289.0,1.382997e+06
2,276.0,1.271089e+06
1,275.0,1.057247e+06
8,282.0,1.044811e+06
3,277.0,1.040093e+06


Vemos de inmediato que existen discrepancias entre los dos totales de ventas. Para el resto de este caso, use este total calculado dinámicamente como la respuesta autorizada.

### Ejercicio 7: (30 min)
Al mirar la documentación, verá que el `subtotal` en la tabla ```salesorderheader``` se calcula a partir de otras tablas en la base de datos. Para validar esta cifra (en lugar de confiar ciegamente en ella), calculemos el `subtotal` manualmente. Utilizando las tablas ```salesorderdetail``` y ```salesorderheader```, calcule las ventas de cada vendedor durante **el año pasado** (2014) y muestre los resultados de los 5 principales vendedores.

**Sugerencia:** Tendrá que ```JOIN``` ```salesorderdetail``` en ```salesorderheader``` para obtener al vendedor, calcular los totales de línea para cada venta usando los descuentos apropiados, luego sumar todos los totales de la línea para obtener la venta total. Querrá usar las cláusulas ```WITH``` nuevamente para mantener las cosas cuerdas.

**Respuesta.**

In [93]:
query = \
"""
WITH sales as (
SELECT soh.salesorderid, ((sod.unitprice-sod.unitpricediscount)*sod.orderqty) total FROM salesorderheader soh JOIN salesorderdetail sod ON sod.salesorderid = soh.salesorderid 
WHERE salespersonid IS NOT NULL AND orderdate BETWEEN '2014-01-01' AND '2014-12-31'
)

SELECT soh.salespersonid, SUM(sales.total) as manual_total FROM salesorderheader soh JOIN sales ON sales.salesorderid = soh.salesorderid GROUP BY soh.salespersonid
"""
res = pd.read_sql(query,engine)
res.sort_values(by="manual_total", ascending=False).head(5)

,salespersonid,manual_total
15,289.0,1.387719e+06
2,276.0,1.281274e+06
1,275.0,1.060208e+06
8,282.0,1.047738e+06
3,277.0,1.042407e+06


### Exercise 8: (20 min)
¿Recuerda cómo mencionamos que los productos se vendían en muchas regiones? Esta es la razón por la que tuvo que trabajar con el valor de `culture` antes para obtener las descripciones del idioma inglés. Para empeorar las cosas, se le dice que las ventas se registran en moneda **local**, por lo que su análisis anterior es defectuoso, y debe convertir todos los montos a USD si desea comparar a los diferentes vendedores de manera justa.

Utilice la tabla `countryregioncurrency` en combinación con las de` salesperson` y `salesterritory` para averiguar el símbolo de moneda relevante para cada uno de los principales vendedores.

**Respuesta.**

### Ejercicio 9: (45 min)

Ahora que tenemos los códigos de moneda asociados con cada vendedor, vuelva a realizar el ejercicio 7 para tener en cuenta el cambio de moneda. Si hay vendedores en el top 5 que no estaban antes, explique por qué.

**Sugerencia:** Las tasas en la tabla ```currencyrate``` siempre van de` FromCurrencyCode = USD` a `ToCurrencyCode = <Código de moneda deseado>`, y se enumeran todos los días. Al calcular los totales de las líneas, use el "AverageRate" 


**Respuesta.**

In [62]:
query9="""
WITH orders
AS (
    SELECT salesorderid,
        sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotal
    FROM salesorderdetail
    GROUP BY salesorderid
    ),
salespersonwithcurrency
AS (
    SELECT a.businessentityid,
        crc.currencycode
    FROM (
        SELECT sp.businessentityid,
            st.countryregioncode
        FROM salesperson sp
        INNER JOIN salesterritory st
            ON sp.territoryid = st.territoryid
        ) a
    INNER JOIN countryregioncurrency crc
        ON crc.countryregioncode = a.countryregioncode
    ),
orderswithcurrency
AS (
    SELECT a.salespersonid,
        a.ordertotal,
        a.orderdate,
        spwc.currencycode
    FROM (
        SELECT *
        FROM orders o
        INNER JOIN salesorderheader soh
            ON o.salesorderid = soh.salesorderid
        WHERE soh.orderdate >= '2014-01-01'
            AND soh.salespersonid != 0
        ) a
    INNER JOIN salespersonwithcurrency spwc
        ON spwc.businessentityid = a.salespersonid
    ),
orderswithcurrencyrate
AS (
    SELECT owc.salespersonid,
        owc.ordertotal,
        owc.ordertotal / cr.averagerate AS ordertotaladjusted,
        owc.orderdate,
        owc.currencycode,
        cr.averagerate
    FROM orderswithcurrency owc
    INNER JOIN currencyrate cr
        ON cr.tocurrencycode = owc.currencycode
    WHERE cr.currencyratedate = owc.orderdate
    ),
salespersontotalsalesadjusted
AS (
    SELECT salespersonid,
        sum(ordertotaladjusted) AS totalsalesadjusted
    FROM orderswithcurrencyrate
    GROUP BY salespersonid
    )
SELECT *
FROM salespersontotalsalesadjusted
ORDER BY totalsalesadjusted DESC LIMIT 5;

"""
runQuery(query9)

,salespersonid,totalsalesadjusted
0,289.0,2.146418e+06
1,276.0,1.271089e+06
2,275.0,1.057247e+06
3,277.0,1.040093e+06
4,290.0,8.443927e+05
